In [1]:
import xml.etree.ElementTree as ET
import glob
import os
import json

In [2]:
def xml_to_yolo_bbox(bbox, w, h):
    # xmin, ymin, xmax, ymax
    x_center = min(((bbox[2] + bbox[0]) / 2) / w,1)
    y_center = min(((bbox[3] + bbox[1]) / 2) / h,1)
    width = min((bbox[2] - bbox[0]) / w,1)
    height = min((bbox[3] - bbox[1]) / h,1)
    return [x_center, y_center, width, height]
def yolo_to_xml_bbox(bbox, w, h):
    # x_center, y_center width heigth
    w_half_len = (bbox[2] * w) / 2
    h_half_len = (bbox[3] * h) / 2
    xmin = int((bbox[0] * w) - w_half_len)
    ymin = int((bbox[1] * h) - h_half_len)
    xmax = int((bbox[0] * w) + w_half_len)
    ymax = int((bbox[1] * h) + h_half_len)
    return [xmin, ymin, xmax, ymax]

In [9]:
classes = []
input_dir = "640_data_augment"
image_dir = f"640_data_augment/images/"
output_dir = "640_data_augment_yolo"
output_labels = f"{output_dir}/labels"
os.mkdir(output_dir)
os.mkdir(output_labels)

FileExistsError: [Errno 17] File exists: '640_data_augment_yolo'

In [7]:
files = glob.glob(os.path.join(f"{input_dir}/annotations/", '*.xml'))
for fil in files:
    basename = os.path.basename(fil)
    filename = os.path.splitext(basename)[0]
    # check if the label contains the corresponding image file
    if not os.path.exists(os.path.join(image_dir, f"{filename}.png")):
        print(f"{filename} image does not exist!")
        continue

    result = []

    # parse the content of the xml file
    tree = ET.parse(fil)
    root = tree.getroot()
    width = int(root.find("size").find("width").text)
    height = int(root.find("size").find("height").text)

    for obj in root.findall('object'):
        label = obj.find("name").text
        # check for new classes and append to list
        if label not in classes:
            classes.append(label)
        index = classes.index(label)
        pil_bbox = [int(x.text) for x in obj.find("bndbox")]
        yolo_bbox = xml_to_yolo_bbox(pil_bbox, width, height)
        # convert data to string
        bbox_string = " ".join([str(x) for x in yolo_bbox])
        result.append(f"{index} {bbox_string}")

    if result:
        # generate a YOLO format text file for each xml file
        with open(os.path.join(output_labels, f"{filename}.txt"), "w", encoding="utf-8") as f:
            f.write("\n".join(result))

# generate the classes file as reference

In [8]:
with open(f'{output_dir}/classes.txt', 'w', encoding='utf8') as f:
    f.write(json.dumps(classes))

# split data

In [10]:
img_dir = f'{input_dir}/images/'
txt_dir = f'{output_dir}/labels/'
#dir = 'drive/MyDrive/no_resize_no_augmentation/'
import os
import random
import shutil

nb_image = len(os.listdir(img_dir))
list_images = []

# for i in os.listdir(img_dir):
#   list_images.append(i)
# random.shuffle(list_images)

# test_images = list_images[:int(len(list_images)/10)]
# train_images = list_images[len(test_images):]
# test_txt = [x[:-3]+ 'txt' for x in test_images]
# train_txt = [x[:-3]+ 'txt' for x in train_images]

for i in os.listdir(img_dir):
  if i[i.find('__')+2:].find('_') == -1 or (i.find('logo') != -1 and i[i.find('__')+2:].count('_') == 1):
    list_images.append(i)
random.shuffle(list_images)

test_images = list_images[:int(len(list_images)/10)]
train_images = list_images[len(test_images):]
test_txt = [x[:-3]+ 'txt' for x in test_images]
train_txt = [x[:-3]+ 'txt' for x in train_images]

In [13]:
os.mkdir(f"{output_dir}/images/")
os.mkdir(f"{output_dir}/images/test/")
os.mkdir(f"{output_dir}/images/train/")
for image in os.listdir(img_dir):
  if image in ['train','test']:
    continue

  if image in test_images or image in train_images:
    image_root = image
  else:
    image_root = image[:image.rfind('_')]+ '.png'

  if image_root in test_images:
    shutil.copy(img_dir + image, f"{output_dir}/images/test/" + image)
  if image_root in train_images:
    shutil.copy(img_dir + image, f"{output_dir}/images/train/" + image)

In [4]:
os.mkdir(txt_dir + 'test/' )
os.mkdir(txt_dir + 'train/')
for txt in os.listdir(txt_dir):
  if txt in ['train','test']:
    continue

  if txt in test_txt or txt in train_txt:
    txt_root = txt
  else:
    txt_root = txt[:txt.rfind('_')]+ '.txt'

  if txt_root in test_txt:
    shutil.move(txt_dir + txt, txt_dir + 'test/' + txt)
  if txt_root in train_txt:
    shutil.move(txt_dir + txt, txt_dir + 'train/' + txt)